# Home assignment 2

You should work on the assignement in groups/teams of 3 participants. Submissions of single students will not be accepted! Please use the Forum in case of doubt in order to find team mates!

Upload your solution as a jupyter notebook to moodle by Tuesday, 7th of January 23:55h. (The deadline is strict)
It is sufficient if one student of each team submits the solution.


You should add comments to your code where necessary and print the relevant results. You should also always test your code on self-chosen examples.

Do not forget to specify the (First_name, Last_name, student_id (matrikelnummer)) of all contributing students in the jupyter notebook here:

Student 1: `Yaping, Chen, 379645`

Student 2: `Qingqing, Yang, 393415`

Student 3: `Yaxin, tang, 402317`

# Implementing GloVe
In this task you will implement the glove algorithm using PyTorch. (One advantage is that you need not calculate gradient by hand, but you can take advantage of the autograd module). The task will require implementation of certain functions, which we look into step-by-step:

Define a function for word to index mapping. Since the model won't be able to take strings as input we will convert them into indices. The function will generate a mapping  w2i which uses words as keys and corresponding indices as values e.g., `w2i['walk'] = 42`. As Preprocessing, remove all punctuations and convert all words to lower case.

In [11]:
import string
from nltk.tokenize import word_tokenize

def word2indexMapping(textfile):
    w2i = {}
    text = [] # sequence of words as they appear in the text after removing punctuations
    
    #Preprocessing
    textfile = textfile.replace('\n', ' ').strip()
    translator = str.maketrans(dict.fromkeys(string.punctuation))
    text = textfile.translate(translator)
    text = text.lower()
    
    #Mapping
    test_list = word_tokenize(text)
    w2i = {k: test_list.index(k) for k in test_list}

    return w2i, text


Define a function for calculating a two dimensional matrix $X_{ij}$ which is the number of times word $j$ occurred in the context of word $i$. The size of the context window $k$ (as a number of words, $k=2$ describes that the context contains the two words before and the two words after the central word) is also an argument of the function. 


In [12]:
import numpy as np

In [13]:
def co_occurrenceFreq(text, w2i, k): # text is a sequence of words ordered as they appear in the text
    # note that there is no notion of sentence here...
    
    X_ij =  np.zeros((len(w2i), len(w2i)))
    
    # write your code snippet here...
    text = text.split(" ")
    tokens = list(w2i.keys())
    token_ids = list(w2i.values())
    context_ids = [] #context for each word
    for center_id in token_ids:
        #left part context for given center
        x = max(0, center_id - k)
        #right part context for given center_id
        y = min(len(text) - 1, center_id + k)
        context_ids.append(list(range(x, y + 1)))
        contexts_len = len(context_ids)
    #print(context_ids)
    count, y = 0, 0 #y represent for index for each center word
    for center_i in tokens:
        for i in range(0,contexts_len): #two demension list interation by i, j
            count = 0
            for j in range(0,len(context_ids[i])):
                if center_i!=text[context_ids[i][j]]:
                    j=j+1
                else: count=count+1
            X_ij[i][y]=count
            i+=1
        y+=1
    return X_ij

Define a GloVe model class with parameters $w$, $\hat w$, $b$ and $\hat b$. For a particular pair of words $i$, $j$, your forward function should implement $w_{i}^{T}\hat w_{j} + b_{i} + \hat b_{j}$. Assume a dimension of embedding to be $d$ which you will specify when creating an instance of the model.

In [14]:
import torch.nn as nn
import torch

In [15]:
class Glove(nn.Module):
    # write your model class here....
    def __init__(self, text, dimension):
        super().__init__()
        self.dimension = dimension
        self.vocab_size = len(text) #text after preprocess, so only unique words contained
        #Word vector matrix V * d
        #All elements are initialized randomly in the range (-0.5,0.5]
        self.w_1 = nn.Parameter((torch.randn(self.vocab_size, dimension) - 0.5) / float(dimension + 1), requires_grad=True)
        self.w_2 = nn.Parameter((torch.randn(self.vocab_size, dimension) - 0.5) / float(dimension + 1), requires_grad=True)
        #bias array size V
        #initialized randomly in the range (-0.5, 0.5]
        self.b_1 = nn.Parameter((torch.randn(self.vocab_size) - 0.5) / float(dimension + 1), requires_grad=True)
        self.b_2 = nn.Parameter((torch.randn(self.vocab_size) - 0.5) / float(dimension + 1), requires_grad=True)

        
    def forward(self, i, j):
        cost = self.w_1[i].dot(self.w_2[j]) + self.b_1[i] + self.b_2[j]#first transpose, then multiply, add
        return cost

    pass

Write a function that implements the weighting function $f(X_{ij})$

$f(x) = (\frac{x}{100})^{\frac{3}{4}}$ if x<100 

$f(x) = 1 $ otherwise

In [16]:
def weightFunction(X_ij, i, j):
    f = 0
    # write your code snippet here
    x_max = 100
    alpha = 0.75
    f = (X_ij[i][j] / x_max) ** alpha if X_ij[i][j] < x_max else 1
    return f

Write a train function to train the model using stochastic gradient descent. Your each training instance would be a word-context pair ($i$, $j$) and the corresponding loss function would be 

$f(X_{ij})(w_{i}^{T}\hat w_{j} + b_{i} + \hat b_{j} - log(1 + X_{ij}))^2$

In [17]:
from torch import optim
import math

def loss_func(X_ij, i, j):
    return weightFunction(X_ij, i, j)*((model.forward(i, j)-math.log(1+X_ij[i][j]))**2)

def train(model, X_ij, learning_rate=0.001, epochs=5):
    opt = optim.Adam(model.parameters(), lr=0.001) # use Adam as your optimizer
    for _ in range(epochs):
        # train across each word-conext pair
        for i in range(len(X_ij)): 
            for j in range(len(X_ij[0])):
                if X_ij[i][j] > 0:     #posive co-occurence frequency
                    loss = loss_func(X_ij, i, j)# calculate loss
                    loss.backward() # backpropagate for every training example
                    opt.step()
                    opt.zero_grad()
    return model.w_1, model.w_2, model.b_1, model.b_2

Write a function to generate embeddings given a word. The embedding of a word with index $i$ would $w_i + \hat w_i$

In [18]:
def getEmbedding(model, word):
    # write your code snippet here...
    index = w2i[word]
    embed = model.w_1[index] + model.w_2[index]
    return embed

Put the individual components together to train a Glove model from the given text file  'shakespeare-caesar.txt' with dimension 200 and a context window of 5.
Manually inspect nearest neigbors for some self-picked words.

In [19]:
with open('data_home_assignment_2\shakespeare-caesar.txt', encoding='latin-1') as f:
    Text = f.read()
w2i, text = word2indexMapping(Text)    
# Test = 'Test for this assighment, it works for a smaller textfile!'
# w2i, text = word2indexMapping(Test)
X_ij = co_occurrenceFreq(text, w2i, k = 5)
model = Glove(text, dimension=200)
print('Original result.....')
print(model.w_1, model.w_2, model.b_1, model.b_2)

Original result.....
Parameter containing:
tensor([[-0.0030, -0.0014, -0.0021,  ..., -0.0021, -0.0115,  0.0059],
        [ 0.0016,  0.0042, -0.0156,  ...,  0.0027, -0.0012, -0.0066],
        [-0.0094, -0.0030, -0.0004,  ..., -0.0125, -0.0015, -0.0007],
        ...,
        [ 0.0021, -0.0030, -0.0038,  ...,  0.0022,  0.0036, -0.0049],
        [-0.0067, -0.0052,  0.0025,  ..., -0.0013, -0.0048, -0.0064],
        [-0.0059, -0.0104, -0.0057,  ..., -0.0048,  0.0015, -0.0057]],
       requires_grad=True) Parameter containing:
tensor([[-0.0006,  0.0004,  0.0017,  ..., -0.0072,  0.0102, -0.0006],
        [ 0.0064, -0.0204, -0.0012,  ..., -0.0045,  0.0030, -0.0006],
        [-0.0035,  0.0024,  0.0054,  ...,  0.0068, -0.0092,  0.0080],
        ...,
        [-0.0037, -0.0058,  0.0030,  ..., -0.0030, -0.0003, -0.0039],
        [ 0.0087, -0.0012,  0.0058,  ...,  0.0026, -0.0051, -0.0025],
        [-0.0074,  0.0025, -0.0082,  ...,  0.0003,  0.0015,  0.0007]],
       requires_grad=True) Parameter con

# Word embeddings with gensim

Gensim is a package which allows you to train word embeddings as well. The task is to take a text file (like 'shakespeare-caesar.txt') and generate embeddings of the vocabulary. You can consult the documentation here - 
https://radimrehurek.com/gensim/models/word2vec.html

In [20]:
import nltk
import re
import gensim.models as gmodels


# get sentece from the raw txt
f = open('data_home_assignment_2\shakespeare-caesar.txt')
raw = f.read()
sent_tokens = nltk.sent_tokenize(raw)

#preprocessing the sentence: strip unnecesary data normalizing cases
sentences=[]
for sent in sent_tokens:
    s = re.split(r'\s+',sent.strip()) #using re to remove \n\n \t and so on
    sentence = [w.lower() for w in s] #normalizing cases
    # print(string.punctuation)
    sentences.append(sentence)
    
model = gmodels.Word2Vec(sentences=sentences)
model.wv.vocab

{'of': <gensim.models.keyedvectors.Vocab at 0x1ba88c8cf08>,
 'caesar': <gensim.models.keyedvectors.Vocab at 0x1ba86451288>,
 'by': <gensim.models.keyedvectors.Vocab at 0x1ba864512c8>,
 'actus': <gensim.models.keyedvectors.Vocab at 0x1ba86451308>,
 'enter': <gensim.models.keyedvectors.Vocab at 0x1ba86451848>,
 'and': <gensim.models.keyedvectors.Vocab at 0x1ba864518c8>,
 'ouer': <gensim.models.keyedvectors.Vocab at 0x1ba86451908>,
 'the': <gensim.models.keyedvectors.Vocab at 0x1ba86451948>,
 'home': <gensim.models.keyedvectors.Vocab at 0x1ba86451348>,
 'you': <gensim.models.keyedvectors.Vocab at 0x1ba86451888>,
 'get': <gensim.models.keyedvectors.Vocab at 0x1ba86451988>,
 'is': <gensim.models.keyedvectors.Vocab at 0x1ba864519c8>,
 'this': <gensim.models.keyedvectors.Vocab at 0x1ba86451a08>,
 'a': <gensim.models.keyedvectors.Vocab at 0x1ba86451a48>,
 'what,': <gensim.models.keyedvectors.Vocab at 0x1ba86451a88>,
 'know': <gensim.models.keyedvectors.Vocab at 0x1ba86451ac8>,
 'not': <gensim.

# Word similarity with pre-trained embeddings

One way of understanding how good the obtained embeddings are is the word similarity task. The file 'sim353.csv' contains a set of word pairs as well as their similarity as judged by humans (e.g., tiger,tiger,10). 
Also we provide a set of pre-trained embeddings in 'embeddings.pickle' in the form of a dictionary with words as keys and embeddings as values. 

Step 1: Consider each word pair in the given file ('sim353.csv') and calculate the cosine similarity between them and then rank the word pairs based on the similarity value

In [21]:
import pickle
import pandas as pd
import numpy as np
from numpy.linalg import norm

#read from pick and csv
pickle_in = open('data_home_assignment_2\embeddings.pickle','rb')
embeddings = pickle.load(pickle_in)
csv_in = pd.read_csv('data_home_assignment_2\sim353.csv',delimiter=',')
df_sim353 = pd.DataFrame(csv_in)

word1 = df_sim353['Word 1']
word2 = df_sim353['Word 2']
sim_human = df_sim353['Human (mean)']
pairlen = len(word1)

# calculate the cosine similarity between word pair, 
# if a word not in the embedding, 
# the similarity will be replaced by the similarity measured by human 
words = embeddings.keys()
cos_sim = []
for i in range(pairlen):
    if word1[i] in words and word2[i] in words:
        a = embeddings[word1[i]]
        b = embeddings[word2[i]]
        # calculate cosin similarity
        cos_sim.append(np.inner(a,b)/(norm(a)*norm(b)))
    else:
        cos_sim.append(sim_human[i]/10) #divid 10 to change it into value in [0,1]

# add a cloumn cosine similarity sorted the table according to it
df = df_sim353.copy(deep=True)
df['cos_sim']= cos_sim
df_cos_sorted = df.sort_values(by=['cos_sim'],ascending=False)
rank = range(1,pairlen+1)
df_cos_sorted['rank (cos.sim.)'] = rank
display(df_cos_sorted)

,Word 1,Word 2,Human (mean),cos_sim,rank (cos.sim.)
2,tiger,tiger,10.00,1.000000,1
90,money,cash,9.08,0.815317,2
30,money,cash,9.15,0.815317,3
38,football,soccer,9.03,0.814274,4
279,championship,tournament,8.36,0.807413,5
...,...,...,...,...,...
138,cup,entity,2.15,0.095603,331
31,professor,cucumber,0.31,0.088859,332
136,cup,artifact,2.92,0.045669,333
88,rooster,voyage,0.62,0.038883,334


Step 2: Rank the word pairs based on the similarity values as determined by humans

In [22]:
df_human_sorted = df.sort_values(by=['Human (mean)'],ascending=False)

df_human_sorted['rank (human)'] = rank
df_human_sorted

,Word 1,Word 2,Human (mean),cos_sim,rank (human)
2,tiger,tiger,10.00,1.000000,1
37,fuck,sex,9.44,0.670561,2
66,midday,noon,9.29,0.665748,3
61,journey,voyage,9.29,0.643200,4
249,dollar,buck,9.22,0.426276,5
...,...,...,...,...,...
88,rooster,voyage,0.62,0.038883,331
87,noon,string,0.54,0.122765,332
85,chord,smile,0.54,0.181856,333
31,professor,cucumber,0.31,0.088859,334


Step 3: Calculate the spearman rank correlation between the two ranked list of word-pairs.

In [23]:
from scipy.stats import spearmanr
# sorted this two lists according to index, each row represents the same word pair
df_cos = df_cos_sorted.sort_index()
df_human = df_human_sorted.sort_index()
data1 = df_human['rank (human)']
data2 = df_cos['rank (cos.sim.)']
spearmanr(data1, data2)

SpearmanrResult(correlation=0.6450611570036899, pvalue=8.391971318047088e-41)

# Text classification with CNNs

You are given a set of news articles which are to be labelled 1 or 0. The data is split into 3 groups (train/test/validation). Each group is further divided into 2 files which consists of the text(ending with \_X.p) and the label (ending with \_y.p) respectively. Each datapoint is a list of words and they are all accumulated in a list forming a list of lists. The label file is a list of labels (0/1). 

We will design a character-level CNN. So the first task would be to obtain a one-hot encoding for the characters. 

In [1]:
# the vocabulary of caracters is provided for your reference.. 
vocabulary = list(""" abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'/\|_@#$%ˆ&*˜‘+-=<>()[]{}""")
len(vocabulary)

69

In [2]:
def characterEncoding(vocabulary):
    c2v = {} # dictionary with key as a character and one-hot encoding as value
    # write your code snippet here..  
    # dictionary that maps every char to an unique int based on position
    c2v = dict((c,i) for i,c in enumerate(vocabulary))
    return c2v
#test
c2v= characterEncoding(vocabulary)
print(c2v)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36, '-': 59, ',': 38, ';': 39, '.': 40, '!': 41, '?': 42, ':': 43, "'": 44, '/': 45, '\\': 46, '|': 47, '_': 48, '@': 49, '#': 50, '$': 51, '%': 52, 'ˆ': 53, '&': 54, '*': 55, '˜': 56, '‘': 57, '+': 58, '=': 60, '<': 61, '>': 62, '(': 63, ')': 64, '[': 65, ']': 66, '{': 67, '}': 68}


Encode a sentence as a 2D matrix with each row representing the one-hot encoding of a character in the sequence.

In [3]:
import numpy as np
import torch
def sentence2tensor(sentence,c2v):
    # Here we assume that all characters in sentences are within the alphabet, we only count the the first 100 characters
    # TODO
    # input: sentence
    # output: a 2D matrix with each row is a vector filled with 0 and 1, and value is where the 1 is.
   
    matrix = []
    for c in sentence:
        if c in c2v:
            vector = (np.zeros((len(c2v)+1),dtype=object))
            vector[c2v[c]] = 1
            matrix.append(vector)

    return torch.LongTensor(matrix)[0:100]

#test
#sentence2tensor("hello world",c2v)

Define a dataset class for feeding data to your model

In [168]:
from torch.utils.data import Dataset, DataLoader

In [4]:
import pickle
import os
import fnmatch
import torch

ListOfFile = os.listdir('data_home_assignment_2/Data/')
print(ListOfFile)
################
# ProcessingData function pre-process data to a list of 2D matrix.It accepts a list of list of words,
# and then change each list of words into a 2D matrix by using sentence2tensor.Returned the first 100 character of each sentence
# since we want to make sure that every matrix shape is the same.
# ** Here we use tensor unsqueeze to add one dimension so that we can feed into the model, we add to second place and first place 
#  is 32, so that we have [32,1,100,69] were 100*69 is the size of matrix, and 32 is the batch size, which every loop there 32 matrices 
# feed into the model.
#
# Input: @file: String indicated "train","test" or "validation" 
# Output: @tensor: tensor type list of 2D matrix
#         @label: tensor type labels
#################
def ProcessingData(file):
    filename = ''
    pre = 'data_home_assignment_2/Data/'
    if file == "test":
        filename = pre+'news_computer_test_X.p'
        labels = pre+'news_computer_test_y.p'
    elif file == "train":
        filename = pre+'news_computer_train_X.p'
        labels = pre+'news_computer_train_y.p'
    elif file == "validation":
        filename = pre+'news_computer_val_X.p'
        labels = pre+'news_computer_val_y.p'
            
    # use pickle to load infomation
    context = pickle.load(open(filename,"rb"))
    lst = []
    for List in context:
        sentence = ''
        for word in List:
            sentence+=word
        #find the shortest sentence and truncate others to same length
        l= sentence2tensor(sentence,c2v)
        lst.append(l)
    
    label = torch.LongTensor(pickle.load(open(labels,"rb"))) 
    tensor = (torch.stack(lst)).unsqueeze(1)
    return tensor.long(),label.long()

a,n = ProcessingData('validation')
print(a.size())

['news_computer_test_X.p', 'news_computer_test_y.p', 'news_computer_train_X.p', 'news_computer_train_y.p', 'news_computer_val_X.p', 'news_computer_val_y.p']
torch.Size([234, 1, 100, 69])


In [5]:
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
###############
#Here we design a NewsDataset inherit Dataset, but most of the processing Data we did outside this function
# 
##############
class NewsDataset(Dataset): 
     def __init__(self, names, labels):
        super(NewsDataset,self).__init__()
        self.names = names
        self.labels = labels
    
     def __len__(self):
        return len(self.names)
    
     def __getitem__(self,index):
        #self.names = Variable(np.random.rand(1,1,100,69))
        #(self.names).unsqueeze(0)
        self.names = Variable(self.names).type(torch.LongTensor)
        self.labels = Variable(self.labels).type(torch.LongTensor)
    
        return self.names[index],self.labels[index]


Define a model class with 2 layers of convolutions each followed by a ReLU unit. The model should have linear layer which maps to the classes. 

In [6]:
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        #self.sample_size = sample_size
        # Defining two 2D convolution layers and followed by ReLU, and one fully connnected layer  
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(69*100*32, 32)
  
      
  # define forward function
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
      
        #flatten x to be one dimension 
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x
   

Define a train function which trains on the train dataset. You can use binary cross entropy as your loss function and Adam as your optimizer.

In [7]:
import torch.optim as optim
from torch.optim import Adam
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
    
def train(model, train_dataset, epochs=5, batch_size=32, learning_rate=0.0001):
  
    #We get data packed in batsch size by using dataloader
    train_dataloader = DataLoader(train_dataset,32)
    # loss
    criterion = nn.CrossEntropyLoss()
    #Optimizer
    optimizer = optim.Adam(model.parameters(),lr= learning_rate)
    for epoch in range(epochs):  
        # loop over the dataset multiple times

        for i, data in enumerate(train_dataloader,0):
            inputs, labels = data
            inputs= inputs.to(torch.float32)
            labels = torch.LongTensor(labels)

            # zero the parameter gradients
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    print('Finished Training')

Define a test function which takes the trained model and test dataset and outputs the accuracy.

In [8]:
########################
# test function calculate the accuracy by feeding data to model and get its predicted output  
#########################
def test(model, test_dataset, batch_size=32):
    
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_dataloader:
            inputs, labels = data
            inputs= inputs.to(torch.float32)
            labels = torch.LongTensor(labels)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the testset: %d %%' % (100 * correct / total))
    
    # write your code snippet here

In [9]:
#################################################################
########### Test DataLoaderis working ######
#################################################################
import torch.optim as optim

# Total has 234 for validation for batch size 32, we will have 8 batches.
t_dataset,t_labels = ProcessingData("validation")
t_data = NewsDataset(t_dataset,t_labels)
ts = DataLoader(t_data, batch_size=32)
for inpu,labels in ts:
    print(inpu.shape)

torch.Size([32, 1, 100, 69])
torch.Size([32, 1, 100, 69])
torch.Size([32, 1, 100, 69])
torch.Size([32, 1, 100, 69])
torch.Size([32, 1, 100, 69])
torch.Size([32, 1, 100, 69])
torch.Size([32, 1, 100, 69])
torch.Size([10, 1, 100, 69])


In [10]:
# putting it altogether
train_data,train_labels = ProcessingData("train")
test_data,test_labels = ProcessingData("test")
  
train_dataset = NewsDataset(train_data,train_labels)    
test_dataset = NewsDataset(test_data,test_labels)

model = Classifier()

train(model, train_dataset)
test(model, test_dataset)

Finished Training
Accuracy of the network on the testset: 62 %


In [1]:
import torch
import sentencepiece as spm
from torchtext import data
from torchtext import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
#import spacy
import time
import matplotlib.pyplot as plt
# Load data from torchtext (identical to what we have in Kaggle)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split()


NameError: name 'TEXT' is not defined